# Pew Research Reproduction

This notebook reproduces the graphs found in
- https://www.pewresearch.org/short-reads/2024/02/15/migrant-encounters-at-the-us-mexico-border-hit-a-record-high-at-the-end-of-2023/

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go

from sqlalchemy import create_engine
from sqlalchemy import text

In [ ]:
DB_CONN = "mysql+pymysql://ts:ts@localhost:3306/us-border"
engine = create_engine(DB_CONN)
conn = engine.connect()

In [ ]:
query = """
SELECT year, month, SUM(count) as count
  FROM cbp_apprehensions_monthly
  WHERE region = "Southern Border"
  GROUP BY year, month
  ORDER BY year, month
"""

res = conn.execute(text(query))
rows = res.mappings().all()

data = []
for row in rows:
  d = dict(row)
  data.append(d)

df = pd.DataFrame(data, columns=["year", "month", "count"])
df["time"] = df["month"] + "-" + df["year"]

fig = px.line(df, x="time", y="count", title='Southern Border Encounters, Monthly')
fig.show(renderer='notebook')

In [ ]:
query = """
SELECT year, count
  FROM nationwide_apprehensions_yearly
"""

res = conn.execute(text(query))
rows = res.mappings().all()

data = []
for row in rows:
  d = dict(row)
  data.append(d)

df = pd.DataFrame(data, columns=["year", "count"])

fig = px.line(df, x="year", y="count", title='Nationwide Encounters, Yearly')
fig.show(renderer='notebook')

In [ ]:
fig = go.Figure()
fig.update_layout(
  title=dict(text="Southern Border Encounters (CBP)"),
)
fig.update_yaxes(range=[0,320000])

def plot(name, color, query, z):
  res = conn.execute(text(query))
  rows = res.mappings().all()

  data = []
  for row in rows:
    d = dict(row)
    data.append(d)

  df = pd.DataFrame(data, columns=["year", "month", "count", "encounter_type"])
  df["time"] = df["month"] + "-" + df["year"]

  fig.add_trace( go.Scatter( x=df["time"], y=df["count"], name=name, line=dict( color=color), zorder=z))

plot(
  name="All", color="#BBBBBB", z=1,
  query = """
  SELECT year, month, SUM(encounter_count) as count
    FROM nationwide_encounters_aor
    WHERE region = "Southwest Land Border"
    GROUP BY year, month
    ORDER BY year, month ASC
  """
)

plot(
  name="Encounters", color="#EF553B", z=0,
  query = """
  SELECT year, month, SUM(encounter_count) as count
    FROM nationwide_encounters_aor
    WHERE region = "Southwest Land Border"
      AND encounter_type != "Inadmissibles"
    GROUP BY year, month
    ORDER BY year, month ASC
  """
)

plot(
  name="Apprehensions", color="#636EFA", z=-1,
  query = """
  SELECT year, month, SUM(encounter_count) as count
    FROM nationwide_encounters_aor
    WHERE region = "Southwest Land Border"
      AND encounter_type = "Apprehensions"
    GROUP BY year, month
    ORDER BY year, month ASC
  """
)

plot(
  name="Inadmissibles (PoE)", color="#FECB52", z=-3,
  query = """
  SELECT year, month, SUM(encounter_count) as count
    FROM nationwide_encounters_aor
    WHERE region = "Southwest Land Border"
      AND encounter_type = "Inadmissibles"
    GROUP BY year, month
    ORDER BY year, month ASC
  """,
)

plot(
  name="Expulsions (COVID)", color="#00CC96", z=-2,
  query = """
  SELECT year, month, SUM(encounter_count) as count
    FROM nationwide_encounters_aor
    WHERE region = "Southwest Land Border"
      AND encounter_type = "Expulsions"
    GROUP BY year, month
    ORDER BY year, month ASC
  """,
)



fig.show(renderer='notebook')

In [ ]:

def plot_group(group, title, where = ""):
  query = f"""
  SELECT year, month, SUM(encounter_count) as count, {group} 
    FROM nationwide_encounters_aor
    {where}
    GROUP BY year, month, {group} 
    ORDER BY year, month, {group}
  """

  res = conn.execute(text(query))
  rows = res.mappings().all()

  data = []
  for row in rows:
    d = dict(row)
    data.append(d)

  df = pd.DataFrame(data, columns=["year", "month", "count", group])
  # df["time"] = pd.to_datetime(dict(year=df["year"], month=df["month"], day=1))
  # df["time"] = pd.date_range('2019-10', freq='M', periods=len(df)).strftime('%Y-%m')
  # df["time"] = pd.to_datetime(df[['year', 'month']], day=15)
  df['time'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str), format='%Y-%b')


  fig = px.bar(df, x="time", y="count", color=group, title=title)
  fig.show(renderer='notebook')


In [ ]:
plot_group("encounter_type", "Encounters, Monthly")

In [ ]:
plot_group("demographic", f"Demographics, Monthly")

In [ ]:
country = "VENEZUELA"
where = f"WHERE citizenship = '{country}'"
plot_group("demographic", f"{country} Demographics, Monthly", where)
plot_group("encounter_type", f"{country} Encounters, Monthly", where)
plot_group("area_of_responsibility", f"{country} by Entry Region, Monthly", where)

In [ ]:
plot_group("citizenship", "Yuma Sector by Citizenship, Monthly", "WHERE area_of_responsibility = 'Yuma Sector'")

In [ ]:
plot_group("region", "Region, Monthly")

In [ ]:
plot_group("area_of_responsibility", "Area of Responsibility, Monthly", "WHERE area_of_responsibility LIKE '%Sector' AND region = 'Southwest Land Border'")
plot_group("area_of_responsibility", "Area of Responsibility, Monthly", "WHERE area_of_responsibility LIKE '%Office' AND region = 'Southwest Land Border'")

In [ ]:
def plot_appreh(group, title, where = ""):
  query = f"""
  SELECT year, month, SUM(count) as count, {group} 
    FROM cbp_apprehensions_monthly
    {where}
    GROUP BY year, month, {group} 
    ORDER BY year, month, {group}
  """

  res = conn.execute(text(query))
  rows = res.mappings().all()

  data = []
  for row in rows:
    d = dict(row)
    data.append(d)

  df = pd.DataFrame(data, columns=["year", "month", "count", group])
  # df["time"] = pd.to_datetime(dict(year=df["year"], month=df["month"], day=1))
  # df["time"] = pd.date_range('2019-10', freq='M', periods=len(df)).strftime('%Y-%m')
  # df["time"] = pd.to_datetime(df[['year', 'month']], day=15)
  df['time'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str), format='%Y-%b')


  fig = px.bar(df, x="time", y="count", color=group, title=title)
  fig.show(renderer='notebook')

In [ ]:
plot_appreh("sector", "Apprehensions by Sector, Monthly", "WHERE region = 'Southern Border'")